In [1]:
from Modular_code.utils import *
#from Modular_code.model import Model
#from Modular_code.train import train_epoch
#from Modular_code.val import val_epoch
from Modular_code.opts import *
from Modular_code.clr import OneCycle
from Modular_code.radam import RAdam

In [ ]:
import pandas as pd
train_label = pd.read_csv(train_label_path)
valid_label = pd.read_csv(valid_label_path)
test_label = pd.read_csv(test_label_path)
train_id,train_classes = create_label(train_label,'./Output/Train') 
test_id,test_classes = create_label(test_label,'./Output/Test') 
valid_id,valid_classes = create_label(valid_label,'./Output/Validation') 

In [ ]:
import pandas as pd
def change_label(eng,bore,frus,conf):
    if(eng == 1 or eng==2):
        if(bore >= 2 or frus >= 2 or conf >= 2):
            return 0
        return 1
    if(eng == 3):
        if(bore >= 2 or frus >= 2 or conf >= 2):
            return 0
        return 2
    if(bore >= 2 or frus >= 2 or conf >= 2 or eng <= 2):
        return 0
def create_label(data_file,path):
    id_1 = data_file['ClipID']
    label = data_file['Engagement']
    bore = data_file.Boredom.values
    conf = data_file.Confusion.values
    frus = data_file.iloc[:,4].values
    id_val = []
    label_val = []
    for id in range(len(id_1)):
        path1 = os.path.join(path,id_1[id][:6])
        id_2 = id_1[id][:-4]
        video_path = os.path.join(path1,id_2)
        if(os.path.exists(video_path)):
            try:
                p = os.path.join(video_path,os.listdir(video_path)[0])
                if(os.path.exists(p)):
                    cap = cv2.VideoCapture(p)
                    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    if(length>=300):
                        id_val.append(os.path.join(video_path,os.listdir(video_path)[0]))
                        label_val.append(change_label(label[id],bore[id],frus[id],conf[id]))
            except:
                pass
    return id_val,label_val

train_id,train_classes = create_label(train_label,'./Output/Train') 
test_id,test_classes = create_label(test_label,'./Output/Test') 
valid_id,valid_classes = create_label(valid_label,'./Output/Validation') 

In [ ]:
def extract_label(id,classes):
    id_new0 = []
    id_new1 = []
    for i,j in enumerate(classes):
        if(j == 0):
            id_new0.append(id[i])
        if(j == 1):
            id_new1.append(id[i])
    return id_new0,id_new1

In [ ]:
train_id = train_id+test_id
train_classes = train_classes+test_classes

In [ ]:
train_id0,train_id1 = extract_label(train_id,train_classes)
import numpy as np
train_id = train_id + 45*train_id0 + 7*train_id1
train_classes = train_classes + np.zeros(len(45*train_id0),dtype = int).tolist() + np.ones(len(7*train_id1) , dtype = int).tolist()

In [ ]:
from collections import Counter
import numpy as np
print(Counter(valid_classes).keys()) # equals to list(set(words))
np.asarray(list(Counter(train_classes).values()))

In [ ]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
im_size = 112
def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])
class video_dataset(Dataset):
    def __init__(self,id,label,sequence_length = 60,transform = None,transform1 = None):
        self.id =id
        self.engagement = label
        self.transform = transform
        self.transform1 = transform1
        self.count = sequence_length
    def __len__(self):
        return len(self.id)
    def __getitem__(self,idx):
        video_path = self.id[idx]
        frames = []
        a = int(300/self.count)
        hp = np.random.randint(0,2)
        aug = np.random.randint(0,2)            
        id1 = np.random.randint(0,a)
        label = self.engagement[idx]
        for i,frame in enumerate(frame_extract(video_path)):
            if(i % a == id1):
                if(label < 2):
                    if(hp == 1):
                        frame = cv2.flip(frame,1)
                frames.append(frame.transpose(2,0,1))
        if(aug == 1):
            if(self.transform1):
                aug_clas = np.random.randint(0,len(self.transform1))
                frames = list(self.transform1[aug_clas].augment_batches(frames,background = False))
        for idx,i in enumerate(frames):
            frames[idx] = self.transform(i.transpose(1,2,0))
        frames = torch.stack(frames)
        #frames = frames[::a,:,:,:]
        frames = frames[:self.count]
#        frames = frames.permute(1,0,2,3)
#        label = label/3
#        print(np.asarray(label)
#        label = one_hot(np.asarray(label),4)
        return frames,label

In [ ]:
im_size = 112
from imgaug import augmenters as iaa

sometimes = lambda aug: iaa.Sometimes(1, aug)
seq = [
    
                    iaa.GaussianBlur((0, 1.0)),
                    iaa.MedianBlur(k=(3, 5)),
                    iaa.MotionBlur(k=3),
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
                
]
train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])
train_data = video_dataset(train_id,train_classes,sequence_length = 100,transform = train_transforms)
val_data = video_dataset(valid_id,valid_classes,sequence_length = 100,transform = train_transforms)
#test_data = video_dataset(test_id,test_classes,sequence_length = 100,transform = train_transforms)

In [ ]:
import cv2
import random
image,label = train_data[300]
#frame = augment(image[10,:,:,:].permute(1,2,0).numpy(),1)
#frame = torch.from_numpy(frame)
print(label)

#for i in range(100):
im_plot(image[50,:,:,:])

In [10]:
from ConvLSTM_pytorch.convlstm import ConvLSTM
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        model = EfficientNet.from_pretrained('efficientnet-b4')
        model._fc = nn.Linear(1792,7)
        model.load_state_dict(torch.load('./effb4emotion.pt'))
        model._fc = Identity()
        for param in model.parameters():
            param.requires_grad = False
        self.model1 = model
        c1 = ConvLSTM(  input_size=(size,size),
                             input_dim= 1792,
                             hidden_dim=[128,512],
                             kernel_size=(3,3),
                             num_layers=2,
                             batch_first = True,
                             bias= True,
                             return_all_layers = False)
        self.c1 = c1
        self.f1 = nn.Linear(512*size*size,2048)
        self.relu = Swish()
        self.fc2 = nn.Linear(2048,512)
        self.fc3 = nn.Linear(512,2)
        self.dp = nn.Dropout(0.4)
    def forward(self, input):
        x = self.model1.extract_features(input)
        x = x.view(-1,100,1792,size,size).detach()
        x = self.c1(x)
        x = torch.mean(x[0][0],dim = 1)
        x = x.view(-1,512*size*size).detach()
        x = self.dp(self.relu(self.f1(x)))
        x = self.dp(self.relu(self.fc2(x)))
        x = self.dp(self.fc3(x))
        return x
#model = Model().to('cuda')
#model = nn.DataParallel(model)
#model.load_state_dict(torch.load('./results/emotionnet-100-bin-convlstm-effb4-112/save.pth')['state_dict'])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
from torch.autograd import Variable
import torchvision.models.resnet as resnet
from efficientnet_pytorch import EfficientNet
class SwishImplementation(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

class Swish(nn.Module):
    @staticmethod
    def forward(x):
        return SwishImplementation.apply(x)

#         Encoder
##############################


class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
#        block = resnet.BasicBlock
#        num_classes = 7
#        layers = [3,4,23,3]
#        model = resnet.ResNet(block, layers, num_classes)
#        model.load_state_dict(torch.load('./checkpoint.pth.tar'))
        model = EfficientNet.from_pretrained('efficientnet-b4')
        model._fc = nn.Linear(1792,7)
        model.load_state_dict(torch.load('./effb4emotion.pt'))
        model._fc = Identity()
        for param in model.parameters():
            param.requires_grad = False
#        model._fc = nn.Linear(1792,7)
#        model.load_state_dict(torch.load('./effb4emotion.pt'))
        self.model = model
        self.bn1 = nn.BatchNorm1d(2048, momentum=0.01)
        #self.final = nn.Sequential(nn.Linear(1280, latent_dim), nn.BatchNorm1d(latent_dim, momentum=0.01))        
    def forward(self, x):
        x = self.model(x)
       # x = x.view(x.size(0), -1)
        return x

##############################
#           LSTM
##############################


class LSTM(nn.Module):
    def __init__(self, latent_dim, num_layers, hidden_dim, bidirectional):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(latent_dim, hidden_dim, num_layers, batch_first=True, bidirectional=bidirectional,dropout = 0.3)
        self.hidden_state = None

    def reset_hidden_state(self):
        self.hidden_state = torch.randn(150, 2, 1024)
    def forward(self, x):
        self.lstm.flatten_parameters()
        x,_ = self.lstm(x, None)
        return x

In [10]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
from torch.autograd import Variable
import torchvision.models.resnet as resnet
from efficientnet_pytorch import EfficientNet
block = resnet.BasicBlock
num_classes = 7
layers = [3,4,23,3]
model = resnet.ResNet(block, layers, num_classes)
model.load_state_dict(torch.load('./checkpoint.pth.tar'))

class SwishImplementation(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

class Swish(nn.Module):
    @staticmethod
    def forward(x):
        return SwishImplementation.apply(x)

#         Encoder
##############################


class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        block = resnet.BasicBlock
        num_classes = 7
        layers = [3,4,23,3]
        model = resnet.ResNet(block, layers, num_classes)
        model.load_state_dict(torch.load('./checkpoint.pth.tar'))
        model = models.resnext50_32x4d(pretrained=True)
        model.fc = nn.Linear(2048,3)
        model.load_state_dict(torch.load('./WACV/best.pth'))
        model = nn.Sequential(*list(model.children())[:-2])
        for param in model.parameters():
            param.requires_grad = False
#        model._fc = nn.Linear(1792,7)
#        model.load_state_dict(torch.load('./effb4emotion.pt'))
        self.model = model
        self.bn1 = nn.BatchNorm1d(2048, momentum=0.01)
        #self.final = nn.Sequential(nn.Linear(1280, latent_dim), nn.BatchNorm1d(latent_dim, momentum=0.01))        
    def forward(self, x):
        x = self.model(x)
       # x = x.view(x.size(0), -1)
        return x

##############################
#           LSTM
##############################


class LSTM(nn.Module):
    def __init__(self, latent_dim, num_layers, hidden_dim, bidirectional):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(latent_dim, hidden_dim, num_layers, batch_first=True, bidirectional=bidirectional,dropout = 0.3)
        self.hidden_state = None

    def reset_hidden_state(self):
        self.hidden_state = torch.randn(150, 2, 1024)
    def forward(self, x):
        self.lstm.flatten_parameters()
        x,_ = self.lstm(x, None)
        return x

In [109]:
import matplotlib.pyplot as plt
class FAN(nn.Module):
    def __init__(
        self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 1024, bidirectional = True):
        super(FAN, self).__init__()
        self.encoder = Encoder()
        self.relu = nn.ELU()
        self.attention1 = nn.Linear(latent_dim,1)
        self.attention2 = nn.Linear(latent_dim*2,1)
        self.sig = nn.Sigmoid()
        self.pool = nn.AdaptiveAvgPool2d(output_size = 1)
        self.dp = nn.Dropout(0.4)
        c1 = ConvLSTM(  input_size=(size,size),
                     input_dim= 2048,
                     hidden_dim=[128,256],
                     kernel_size=(3,3),
                     num_layers=2,
                     batch_first = True,
                     bias= True,
                     return_all_layers = False)
        self.c1 = c1

        self.output_layers1 = nn.Sequential(
            nn.Linear(latent_dim*2, 1024),nn.ELU(),nn.Dropout(0.4),nn.Linear(1024,4),nn.Dropout(0.4))
    def forward(self, x):
        batch_size, seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        x1 = self.encoder(x)
        x = self.pool(x1)
        x = x.view(batch_size,seq_length,2048)
        v1 = []
        for i in range(x.shape[0]):
            alpha = self.sig(self.attention1(self.dp(x[i])))
            x1 = torch.sum(x[i]*alpha,dim = 0)
            a1 = torch.sum(alpha , dim = 0)
            x1 = torch.div(x1,a1)
            x11 = x1.unsqueeze(dim = 0).repeat(seq_length,1)
            x3 = torch.cat([x[i],x11],dim = 1)
            betas = self.sig(self.attention2(self.dp(x3)))
            v1.append(x3*alpha*betas)
        v1 = torch.stack(v1)
        v1 = torch.sum(v1,dim = 1)
        v1 = torch.div(v1,torch.sum(alpha*betas,dim = 0))
        x = self.c1(x1)
        x = x[0][0].view(-1,256*size*size)
        #att = self.dp(self.sig(self.attention(x))).view(-1,100)
        x = torch.mean(x.view(-1,100,256*size*size),dim = 1)
        print(x.shape)
        return v1

In [110]:
from torchvision import models
model = FAN(4).cuda()
model = nn.DataParallel(model)
#model_dict = model.state_dict()
#pretrained_dict = torch.load('./results/face-100-comp-fan/save.pth')['state_dict']
#pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
#model_dict.update(pretrained_dict)
model(image.unsqueeze(0).cuda())

IndexError: Caught IndexError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/sanjaymoto75/.conda/envs/env/lib/python3.7/site-packages/torch/nn/parallel/parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "/home/sanjaymoto75/.conda/envs/env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 547, in __call__
    result = self.forward(*input, **kwargs)
  File "<ipython-input-109-cfa5d0a0062c>", line 44, in forward
    x = self.c1(x1)
  File "/home/sanjaymoto75/.conda/envs/env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 547, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/sanjaymoto75/data/home/sanjaymoto75/attention/ConvLSTM_pytorch/convlstm.py", line 128, in forward
    seq_len = input_tensor.size(1)
IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)


In [10]:
size = 3
from torchvision import models
from efficientnet_pytorch import EfficientNet
class SwishImplementation(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

class Swish(nn.Module):
    @staticmethod
    def forward(x):
        return SwishImplementation.apply(x)
from ConvLSTM_pytorch.convlstm import ConvLSTM
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        model = EfficientNet.from_pretrained('efficientnet-b4')
        model._fc = nn.Linear(1792,7)
        model.load_state_dict(torch.load('./effb4emotion.pt'))
        model._fc = Identity()
        for param in model.parameters():
            param.requires_grad = False
        self.model1 = model
        c1 = ConvLSTM(  input_size=(size,size),
                             input_dim= 1792,
                             hidden_dim=[128,256],
                             kernel_size=(3,3),
                             num_layers=2,
                             batch_first = True,
                             bias= True,
                             return_all_layers = False)
        self.c1 = c1
#        self.attention = nn.Linear(256*size*size,1)
        self.f1 = nn.Linear(256*size*size+256,512)
        self.relu = nn.ReLU()
#        self.fc2 = nn.Linear(2048,512)
        self.att_red = nn.Linear(1792,256)
        self.fc4 = nn.Linear(512,4)
        self.dp = nn.Dropout(0.4)
        self.sig = nn.Sigmoid()
        self.pool = nn.AdaptiveAvgPool2d(output_size = 1)
        self.attention1 = nn.Linear(1792,1)
#        self.beta_attention = nn.Linear(4096,1)
    def forward(self, input):
        input = input.view(-1,3,112,112)
        x = self.model1.extract_features(input)
        x_pool = self.pool(x).reshape(-1,100,1792)
        x = x.view(-1,100,1792,size,size)
        x = self.c1(x)
        x = x[0][0].view(-1,256*size*size)
        x = x.view(-1,100,256*size*size)
        #x1 = []
        x2 = []
        v1 = []
        for i in range(x_pool.shape[0]):            
            alpha = self.sig(self.attention1(self.dp(x_pool[i])))
            #x1.append(torch.div(torch.sum(x[i]*att[i].reshape(-1,1),dim = 0),torch.sum(att[i])))
            x2.append(torch.div(torch.sum(x_pool[i]*alpha.reshape(-1,1),dim = 0),torch.sum(alpha,dim = 0)))
        x2 = torch.stack(x2)
        x2 = self.relu(self.att_red(x2))
#        v1 = torch.stack(v1)
#        v1 = torch.sum(v1,dim = 1)
#        v1 = torch.div(v1,torch.sum(alpha*betas,dim = 0))
        x1 = torch.mean(x,dim = 1)
        x = torch.cat([x1,x2],dim = 1)
#        x = self.dp(self.relu(self.fc3(x)))
        #x = torch.div(x,torch.sum(att,dim = 1))
        x = self.dp(self.relu(self.f1(x)))
#        x = self.dp(self.relu(self.fc2(x)))
        x = self.dp(self.fc4(x))
        x = 
        return x
model = Model().to('cuda')
model = nn.DataParallel(model)

SyntaxError: invalid syntax (<ipython-input-10-0222d3388597>, line 85)

In [10]:
size = 4
from torchvision import models
from ConvLSTM_pytorch.convlstm import ConvLSTM
import torch
import torch.nn.functional as F
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained=True)
        model.fc = nn.Linear(2048,3)
        model.load_state_dict(torch.load('./WACV/best.pth'))
        model1 = nn.Sequential(*list(model.children())[:-2])
        for param in model1.parameters():
            param.requires_grad = False
        self.model1 = model1
        c1 = ConvLSTM(  input_size=(size,size),
                             input_dim= 2048,
                             hidden_dim=[128,256],
                             kernel_size=(3,3),
                             num_layers=2,
                             batch_first = True,
                             bias= True,
                             return_all_layers = False)
        self.c1 = c1
#        self.f1 = nn.Linear(256*size*size,512)
        self.f1 = nn.Linear(256*size*size+512,512)
        self.relu = nn.LeakyReLU()
        self.att_red = nn.Linear(2048,512)
        self.fc4 = nn.Linear(512,4)
        self.dp = nn.Dropout(0.4)
        self.sig = nn.Sigmoid()
        self.pool = nn.AdaptiveAvgPool2d(output_size = 1)
        self.attention1 = nn.Linear(2048,1)        
        self.r = nn.ReLU()

    def forward(self, input):
        input = input.view(-1,3,112,112)
        x = self.model1(input)
        x_pool = self.relu(self.pool(x)).reshape(-1,100,2048)
        x = x.view(-1,100,2048,size,size)
        x = self.c1(x)
        x = x[0][0].view(-1,256*size*size)
        x = x.view(-1,100,256*size*size)
        x2 = []
        v1 = []
        for i in range(x_pool.shape[0]):            
            alpha = self.sig(self.attention1(self.dp(x_pool[i])))
#            x1.append(torch.div(torch.sum(x[i]*att[i].reshape(-1,1),dim = 0),torch.sum(att[i])))
            x2.append(torch.div(torch.sum(x_pool[i]*alpha.reshape(-1,1),dim = 0),torch.sum(alpha,dim = 0)))
        x2 = torch.stack(x2)
        x2 = self.relu(self.att_red(x2))
        x = torch.mean(x,dim = 1)
        x = torch.cat([x,x2],dim = 1)
        x1 = self.dp(self.relu(self.f1(x)))
        x = self.dp(self.fc4(x1))
        return x
model = Model().to('cuda')
#model = nn.DataParallel(model)
#model.load_state_dict(torch.load('./results/face-wacv-100-comp-convlstm+att/save.pth')['state_dict'])

In [11]:
import numpy as np
im_size = 112
model(torch.from_numpy(np.empty((2,100,3,im_size,im_size))).type(torch.cuda.FloatTensor))

tensor([[0.0000, 0.0161, 0.0544, 0.0058],
        [0.0685, 0.0000, 0.0000, 0.0000]], device='cuda:0',
       grad_fn=<FusedDropoutBackward>)

In [12]:
arch = 'face-wacv-100-comp-convlstm+att-1'
result_path = os.path.join('./results',arch)
from tensorboardX import SummaryWriter
writer = SummaryWriter()
os.makedirs(result_path,exist_ok = True)
train_logger = Logger(
            os.path.join(result_path, 'train{}.log'.format(arch)),
            ['epoch', 'loss', 'acc', 'lr'])
train_batch_logger = Logger(
            os.path.join(result_path, 'train_batch{}.log'.format(arch)),
            ['epoch', 'batch', 'iter', 'loss', 'acc', 'lr'])    
val_logger = Logger(
            os.path.join(result_path, 'val{}.log'.format(arch)), ['epoch', 'loss', 'acc'])

In [13]:
def set_lr(optimizer, lr):
    for group in optimizer.param_groups:
        group['lr'] = lr

In [14]:
import torch
from torch.autograd import Variable
import time
import os
import sys
import os
def train_epoch(epoch, num_epochs, data_loader, model, criterion, optimizer, epoch_logger, batch_logger, batch_size , onecyc , writer , result_path):
    print('Training Epoch {}'.format(epoch))
    model.train()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracies = AverageMeter()
    start_time = time.time()
    end_time = time.time()
    t = []
    for i, (inputs, targets) in enumerate(data_loader):
        if(targets.shape[0]<batch_size):
            continue
        data_time.update(time.time() - end_time)
        if torch.cuda.is_available():
            targets = targets.type(torch.cuda.LongTensor)
            inputs = inputs.cuda()
        outputs = model(inputs)
        
#        targets = torch.nn.functional.one_hot(targets,4)
#        print(targets)
        loss = torch.mean(criterion(outputs, targets))
#        _,targets = torch.max(targets,1)
        acc = calculate_accuracy(outputs, targets)
        losses.update(loss.item(), inputs.size(0))
        accuracies.update(acc, inputs.size(0))
        lr,_ = onecyc.calc()
        update_lr(optimizer, lr)
        optimizer.zero_grad()
#        with amp.scale_loss(loss, optimizer) as scaled_loss:
#            scaled_loss.backward()
        loss.backward()
        optimizer.step()
        t.extend(list(targets.detach().cpu().numpy()))
        del inputs,targets,outputs
        batch_time.update(time.time() - end_time)
        end_time = time.time()
        batch_logger.log({
            'epoch': epoch,
            'batch': i + 1,
            'iter': (epoch - 1) * len(data_loader) + (i + 1),
            'loss': losses.val,
            'acc': accuracies.val,
            'lr': optimizer.param_groups[0]['lr']
        })
        writer.add_scalar('data/acc', accuracies.val, (epoch - 1) * len(data_loader) + (i + 1))
        writer.add_scalar('data/loss', losses.val, (epoch - 1) * len(data_loader) + (i + 1))        
        sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d / %d] [Time %.2f %.2f] [Data %.2f %.2f] [Loss: %f, Acc: %.2f%%]"
                % (
                    epoch,
                    num_epochs,
                    i,
                    len(data_loader),
                    batch_time.val,
                    batch_time.avg,
                    data_time.val,
                    data_time.avg,
                    losses.avg,
                    accuracies.avg
                    )
                )
    print('\nEpoch time {} mins'.format((end_time-start_time)/60))
    epoch_logger.log({
        'epoch': epoch,
        'loss': losses.avg,
        'acc': accuracies.avg,
        'lr': optimizer.param_groups[0]['lr']
    })
    writer.add_scalar('data/acc_epoch', accuracies.avg, epoch)
    writer.add_scalar('data/loss_epoch', losses.avg, epoch)        
        
    save_file_path = os.path.join(result_path,'save.pth')
    states = {
        'state_dict': model.state_dict()
    }
    torch.save(states, save_file_path)
def test(epoch,model, data_loader ,criterion, batch_size, result_path,best_acc = 0 , logger = None ):
    print('Testing')
    if(epoch == 1):
        best_acc = 0
    model.eval()
    losses = AverageMeter()
    accuracies = AverageMeter()
    pred = []
    true = []
    for i, (inputs, targets) in enumerate(data_loader):
        if torch.cuda.is_available():
            targets = targets.cuda()
            inputs = inputs.cuda()
        outputs = model(inputs)
        _,p = torch.max(outputs,1)
        loss = criterion(outputs, targets)
#        _,targets = torch.max(targets,1)
        acc = calculate_accuracy(outputs, targets)
        true += (targets.type(torch.cuda.LongTensor)).detach().cpu().numpy().reshape(len(targets)).tolist()
        pred += p.detach().cpu().numpy().reshape(len(p)).tolist()
        losses.update(loss.item(), inputs.size(0))
        accuracies.update(acc, inputs.size(0))
        sys.stdout.write(
                "\r[Batch %d / %d]  [Loss: %f, Acc: %.2f%%]"
                % (
                    i,
                    len(data_loader),
                    losses.avg,
                    accuracies.avg
                    )
                )
    if(logger):
        logger.log({'epoch': epoch, 'loss': losses.avg, 'acc': accuracies.avg})
    print('\nAccuracy {}'.format(accuracies.avg))
    if(accuracies.avg>best_acc):
        best_acc = accuracies.avg
        result_path = os.path.join(result_path,'best.pth')
        state = {
        'acc':best_acc,
         'state':model.state_dict()
        }
        torch.save(state,result_path)
    return true,pred,best_acc

In [65]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
class WeightedKappaLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, preds, true,nb_classes = None):
        #step1 confusion matrix
        nb_classes = preds.shape[1]
        _,pred = torch.max(preds,1)
        confusion_matrix = torch.empty([nb_classes, nb_classes],requires_grad = True)
        confusion_matrix.requires_grad = True
        for t, p in zip(pred.view(-1), true.view(-1)):
            confusion_matrix[p.long(), t.long()] += 1
        weights = torch.empty([nb_classes,nb_classes],requires_grad = True)
        weights.requires_grad = True
        #weight matrix
        for i in range(len(weights)):
            for j in range(len(weights)):
                
                weights[i][j] = float(((i-j)**2)/(len(weights)-1)**2)
        #Histograms
        true_hist= torch.empty([nb_classes],requires_grad = True)
        true_hist.requires_grad = True
        for item in true: 
            true_hist[item]+=1
        pred_hist=torch.empty([nb_classes],requires_grad = True)
        pred_hist.requires_grad = True
        for item in pred: 
            pred_hist[int(item)]+=1
        E = torch.ger(true_hist,pred_hist)
        E = E/E.sum()
        confusion_matrix = confusion_matrix/confusion_matrix.sum()
        num = (confusion_matrix*weights).sum()
        den = (E*weights).sum()
        return num/den

In [33]:
class sanchitLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.sig = nn.Tanh()
        self.soft = nn.LogSoftmax()
    def forward(self, pred, true):
        clases = torch.arange(0,pred.shape[1],1).reshape(1,-1).repeat(pred.shape[0],1)
        if(torch.cuda.is_available()):
            clases = clases.cuda()
        clases = (abs(clases-true.reshape(-1,1))).type(torch.cuda.DoubleTensor)
        clases = clases/(clases.shape[1]-1)
        if(torch.cuda.is_available()):
            dis = torch.log(torch.cosh(clases+1e-12)).type(torch.cuda.DoubleTensor)
            pred = self.soft(pred).type(torch.cuda.DoubleTensor)
        else:
            dis = torch.log(torch.cosh(clases+1e-12)).type(torch.cuda.DoubleTensor)
            pred = self.soft(pred).type(torch.DoubleTensor)
        return torch.mean(-1*torch.sum(dis * pred,axis = 1),axis = 0)
s = sanchitLoss()
s(torch.from_numpy(np.asarray([0.1,0.2,0.3,0.4])).unsqueeze(dim = 0).type(torch.cuda.FloatTensor),torch.from_numpy(np.asarray([3])).type(torch.cuda.LongTensor))

/home/sanjaymoto75/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


tensor(1.0417, device='cuda:0', dtype=torch.float64)

In [39]:
import torch
import numpy as np
from torch import nn
class sanchitLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.sig = nn.Tanh()
        self.soft1 = nn.LogSoftmax()
        self.soft = nn.Softmax()
        self.nll = nn.NLLLoss()
    def forward(self, pred, true):
        clases = torch.arange(0,pred.shape[1],1).reshape(1,-1).repeat(pred.shape[0],1)
        if(torch.cuda.is_available()):
            clases = clases.cuda()
        clases = (abs(clases-true.reshape(-1,1))).type(torch.cuda.DoubleTensor)
        dis = clases/(clases.shape[1]-1)
        #print(self.nll(self.soft1(pred),true))
        if(torch.cuda.is_available()):
#            dis = self.sig(clases).type(torch.cuda.DoubleTensor)
            pred = self.soft(pred).type(torch.cuda.DoubleTensor)
        else:
            #dis = self.sig(clases).type(torch.cuda.DoubleTensor)
            pred = self.soft(pred).type(torch.DoubleTensor)
        return torch.mean(torch.sum(dis * pred,axis = 1).type(torch.cuda.DoubleTensor),axis = 0)
s = sanchitLoss()
s(torch.from_numpy(np.asarray([0,0.2,0.3,0.5])).unsqueeze(dim = 0).type(torch.cuda.FloatTensor),torch.from_numpy(np.asarray([3])).type(torch.cuda.LongTensor))

/home/sanjaymoto75/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.4338, device='cuda:0', dtype=torch.float64)

In [ ]:
from sklearn.metrics import confusion_matrix
model = Model().to('cuda')
#model.load_state_dict(torch.load('./results/face-wacv-100-comp-convlstm+att/save.pth')['state_dict'])
model = nn.DataParallel(model)
class LogCoshLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, y_t, y_prime_t):
        ey_t = y_t - y_prime_t
        return torch.mean(torch.log(torch.cosh(ey_t + 1e-12)))
class XTanhLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, y_t, y_prime_t):
        ey_t = y_t - y_prime_t
        return torch.mean(ey_t * torch.tanh(ey_t))

class XSigmoidLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, y_t, y_prime_t):
        ey_t = y_t - y_prime_t
        return torch.mean(2 * ey_t / (1 + torch.exp(-ey_t)) - ey_t)
from imblearn.metrics import classification_report_imbalanced
from apex import amp
lr = 1e-3
batch_size = 64
train_loader = DataLoader(train_data,batch_size = batch_size , num_workers = 8,shuffle = True)
val_loader = DataLoader(val_data,batch_size = batch_size, num_workers = 8)
#test_loader = DataLoader(test_data,batch_size = batch_size , num_workers = 16)
class_weights = torch.from_numpy(np.asarray([2,2,1,1])).type(torch.FloatTensor).cuda()
optimizer = RAdam(model.parameters(), lr= lr,weight_decay = 1e-3)
#model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
criterion = nn.CrossEntropyLoss(weight = class_weights , reduction = 'none').cuda()
best_acc = 0
beta = 0.9999
gamma = 0.5
samples_per_cls = [544,1065,2615,2493]
loss_type = "focal"
no_of_classes = 4
num_epochs = 15
onecyc = OneCycle(len(train_loader)*num_epochs, lr)
for epoch in range(1,num_epochs+1):
    train_epoch(epoch,num_epochs,train_loader,model,criterion,optimizer,train_logger,train_batch_logger,batch_size,onecyc,writer,result_path)
    true,pred,best_acc= test(epoch,model,val_loader,criterion,batch_size,result_path,best_acc)
    cm1 = confusion_matrix(true,pred)
    print('Comb\n',cm1)
    print(classification_report_imbalanced(true, pred))

Training Epoch 1
[Epoch 1/15] [Batch 168 / 170] [Time 1.37 1.58] [Data 0.12 0.18] [Loss: 1.789602, Acc: 30.13%]
Epoch time 4.461353965600332 mins
Testing
[Batch 21 / 23]  [Loss: 1.537648, Acc: 18.75%]
Accuracy 18.75
Comb
 [[  3  19   0   1]
 [  7 123   9   4]
 [ 46 599  84  75]
 [ 57 279  48  54]]
                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      0.13      0.92      0.04      0.35      0.11        23
          1       0.12      0.86      0.29      0.21      0.50      0.26       143
          2       0.60      0.10      0.91      0.18      0.31      0.09       804
          3       0.40      0.12      0.92      0.19      0.34      0.10       438

avg / total       0.48      0.19      0.85      0.18      0.34      0.11      1408

Training Epoch 2
[Epoch 2/15] [Batch 168 / 170] [Time 1.41 1.58] [Data 0.13 0.18] [Loss: 1.603441, Acc: 41.24%]
Epoch time 4.462957179546356 mins
Testing
[Batch 21 / 23]  [Loss: 1.469799, Acc: 34.02%]

In [20]:
## MSE loss pipeline
import torch
from torch.autograd import Variable
import time
import os
import sys
import os
def train_epoch(epoch, num_epochs, data_loader, model, criterion, optimizer, epoch_logger, batch_logger, batch_size , onecyc , writer , result_path):
    print('Training Epoch {}'.format(epoch))
    model.train()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracies = AverageMeter()
    start_time = time.time()
    end_time = time.time()
    t = []
    for i, (inputs, targets) in enumerate(data_loader):
        data_time.update(time.time() - end_time)
        if(inputs.size(0)<batch_size):
            continue
        if torch.cuda.is_available():
            targets = targets.type(torch.cuda.LongTensor)
            inputs = inputs.cuda()
        outputs = model(inputs)
        loss = torch.mean(criterion(outputs,targets.type(torch.cuda.LongTensor)))
#        loss2 = criterion((output).type(torch.cuda.FloatTensor), (targets/3).type(torch.cuda.FloatTensor))
#        loss = (1-lambda)*loss1 + lambda*loss
#        loss = loss2 + loss1
#        loss = loss1
        acc = calculate_accuracy(outputs, targets.type(torch.cuda.LongTensor))
        losses.update(loss.item(), inputs.size(0))
        accuracies.update(acc, inputs.size(0))
        lr,_ = onecyc.calc()
        update_lr(optimizer, lr)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_time.update(time.time() - end_time)
        end_time = time.time()
        batch_logger.log({
            'epoch': epoch,
            'batch': i + 1,
            'iter': (epoch - 1) * len(data_loader) + (i + 1),
            'loss': losses.val,
            'acc': accuracies.val,
            'lr': optimizer.param_groups[0]['lr']
        })
        writer.add_scalar('data/acc', accuracies.val, (epoch - 1) * len(data_loader) + (i + 1))
        writer.add_scalar('data/loss', losses.val, (epoch - 1) * len(data_loader) + (i + 1))  
        sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d / %d] [Time %.2f %.2f] [Data %.2f %.2f] [Loss: %f, Acc: %.2f%%]"
                % (
                    epoch,
                    num_epochs,
                    i,
                    len(data_loader),
                    batch_time.val,
                    batch_time.avg,
                    data_time.val,
                    data_time.avg,
                    losses.avg,
                    accuracies.avg))
    print('\nEpoch time {} mins'.format((end_time-start_time)/60))
    epoch_logger.log({
        'epoch': epoch,
        'loss': losses.avg,
        'acc': accuracies.avg,
        'lr': optimizer.param_groups[0]['lr']
    })
    writer.add_scalar('data/acc_epoch', accuracies.avg, epoch)
    writer.add_scalar('data/loss_epoch', losses.avg, epoch)        
        
    save_file_path = os.path.join(result_path,'save.pth')
    states = {
        'state_dict': model.state_dict()
    }
    torch.save(states, save_file_path)
    return t
def test(epoch,model, data_loader ,criterion, batch_size, result_path,best_acc = 0 , logger = None ):
    print('Testing')
#    criterion = nn.L1Loss().cuda()
    if(epoch == 1):
        best_acc = 0
    model.eval()
    losses = AverageMeter()
    accuracies = AverageMeter()
    pred = []
    true = []
    count = 0
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(data_loader):
            if(inputs.size(0)*2<batch_size):
                continue
            if torch.cuda.is_available():
                targets = targets.cuda().type(torch.cuda.FloatTensor)
                inputs = inputs.cuda()
            outputs = model(inputs)
            loss = torch.mean(criterion(outputs, targets.type(torch.cuda.LongTensor)))
            acc = calculate_accuracy(outputs,targets.type(torch.cuda.LongTensor))
            _,p = torch.max(outputs,1) 
            true += (targets.type(torch.cuda.LongTensor)).detach().cpu().numpy().reshape(len(targets)).tolist()
            pred += p.detach().cpu().numpy().reshape(len(p)).tolist()
            losses.update(loss.item(), inputs.size(0))
            accuracies.update(acc, inputs.size(0))
            sys.stdout.write(
                    "\r[Batch %d / %d]  [Loss: %f, Acc: %.2f%%]"
                    % (
                        i,
                        len(data_loader),
                        losses.avg,
                        accuracies.avg
                        )
                    )
        if(logger):
            logger.log({'epoch': epoch, 'loss': losses.avg, 'acc': accuracies.avg})
        print('\nAccuracy {}'.format(accuracies.avg))
        if(accuracies.avg>best_acc):
            best_acc = accuracies.avg
            result_path = os.path.join(result_path,'best.pth')
            state = {
            'acc':best_acc,
             'state':model.state_dict()
            }
            torch.save(state,result_path)
    return true,pred,best_acc